In [1]:
import os
if '_fixed' not in locals():
    _fixed = os.chdir(os.path.dirname(os.getcwd()))
import json
import omnifig as fig
from tqdm.notebook import tqdm
from ludwig.util import PromptTemplate
from ludwig.tictactoe import TakeTheMiddle
from ludwig.util import vllm_Client
from ludwig.util.clients import SAIA_Client
from ludwig.util import extract_code_blocks

In [2]:
client = fig.create_config('secrets/saia', _type='saia', model_name=None).create()

| CREATING . type='saia'
| model: None (by default)
| use-chat-completion: False (by default)
| tool-style: 'pythonic' (by default)
| chat-template-path: None (by default)
| chat-template: None (by default)
| enable-thinking: False (by default)
| add-generation-prompt: True (by default)
| continue-final-message: False (by default)
| max-tokens: None (by default)
| seed: None (by default)
| temperature: None (by default)
| top-p: None (by default)
| grammar: None (by default)
| raise-length-limit: True (by default)
| system-message: None (by default)
| message-parser: None (by default)
| debug-log: False (by default)


In [3]:
models = client.available_models()
models

[{'id': 'meta-llama-3.1-8b-instruct',
  'created': 1755111540,
  'object': 'model',
  'owned_by': 'chat-ai',
  'input': ['text'],
  'output': ['text'],
  'name': 'Meta Llama 3.1 8B Instruct',
  'demand': 0,
  'status': 'ready'},
 {'id': 'openai-gpt-oss-120b',
  'created': 1755111540,
  'object': 'model',
  'owned_by': 'chat-ai',
  'input': ['text'],
  'output': ['text'],
  'name': 'OpenAI GPT OSS 120B',
  'demand': 0,
  'status': 'ready'},
 {'id': 'gemma-3-27b-it',
  'created': 1755111540,
  'object': 'model',
  'owned_by': 'chat-ai',
  'input': ['text', 'image'],
  'output': ['text'],
  'name': 'Gemma 3 27B Instruct',
  'demand': 0,
  'status': 'ready'},
 {'id': 'qwen3-32b',
  'created': 1755111540,
  'object': 'model',
  'owned_by': 'chat-ai',
  'input': ['text'],
  'output': ['text', 'thought'],
  'name': 'Qwen 3 32B',
  'demand': 0,
  'status': 'ready'},
 {'id': 'qwen3-235b-a22b',
  'created': 1755111540,
  'object': 'model',
  'owned_by': 'chat-ai',
  'input': ['text'],
  'output'

In [4]:
from tabulate import tabulate

print(tabulate([x['id'], x['name']] for x in models))

-----------------------------------  -----------------------------------
meta-llama-3.1-8b-instruct           Meta Llama 3.1 8B Instruct
openai-gpt-oss-120b                  OpenAI GPT OSS 120B
gemma-3-27b-it                       Gemma 3 27B Instruct
qwen3-32b                            Qwen 3 32B
qwen3-235b-a22b                      Qwen 3 235B A22B 2507
llama-3.3-70b-instruct               Meta Llama 3.3 70B Instruct
qwen2.5-vl-72b-instruct              Qwen 2.5 VL 72B Instruct
medgemma-27b-it                      MedGemma 27B Instruct
qwq-32b                              Qwen QwQ 32B
deepseek-r1                          DeepSeek R1 0528
deepseek-r1-distill-llama-70b        DeepSeek R1 Distill Llama 70B
mistral-large-instruct               Mistral Large Instruct
qwen2.5-coder-32b-instruct           Qwen 2.5 Coder 32B Instruct
internvl2.5-8b                       InternVL2.5 8B MPO
teuken-7b-instruct-research          Teuken 7B Instruct Research
codestral-22b                        C

In [3]:

from openai import OpenAI

base_url = "https://chat-ai.academiccloud.de/v1"
model = "meta-llama-3.1-8b-instruct" # Choose any available model
model = 'openai-gpt-oss-120b'
  
# Start OpenAI client
client = OpenAI(
    api_key = fig.create_config('secrets/saia').pull('api-key', silent=True),
    base_url = base_url
)
  
# Get response
chat_completion = client.chat.completions.create(
        messages=[{"role":"system","content":"You are a helpful assistant"},{"role":"user","content":"How tall is the Eiffel tower?"}],
        model= model,
    )

In [4]:
chat_completion.to_dict()

{'id': 'chatcmpl-751d6b76a40b438080ba89516c9a9630',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'The Eiffel\u202fTower stands **324\u202fm** tall (about **1,063\u202fft**) when you include its antenna.  \n- Height to the roof (the top of the iron structure) is **300\u202fm** (≈\u202f984\u202fft).  \n- It was originally completed in 1889 at 312\u202fm (≈\u202f1,024\u202fft) before the radio antenna was added.',
    'refusal': None,
    'role': 'assistant',
    'annotations': None,
    'audio': None,
    'function_call': None,
    'tool_calls': [],
    'reasoning_content': 'The user asks: "How tall is the Eiffel tower?" Straightforward question. Provide height in meters and feet. Perhaps mention different heights (original, to top, etc.). Answer concisely.'},
   'stop_reason': None}],
 'created': 1755077914,
 'model': 'openai-gpt-oss-120b',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': None,
 'usage': 

In [5]:
print(chat_completion.choices[0].message.content)

The Eiffel Tower stands **324 m** tall (about **1,063 ft**) when you include its antenna.  
- Height to the roof (the top of the iron structure) is **300 m** (≈ 984 ft).  
- It was originally completed in 1889 at 312 m (≈ 1,024 ft) before the radio antenna was added.


In [ ]:
model_name = 

In [2]:
task = TakeTheMiddle(obs_rep='moves')
task.prepare()
task.total_questions

270

In [53]:
client = vllm_Client(addr='8002').prepare()
client.model_name

'meta-llama/Llama-3.3-70B-Instruct'

In [ ]:
ctx = {
	'task': task.description(),
	'context': task.context(),
}

ctx['task'] = "We're playing tic-tac-toe, and, given the current game state, I have a question about whether I should play in the middle or not."

samples = [task.ask_dev(i) for i in range(10)]
ctx.update({f'{key}{i}': value for i, sample in enumerate(samples) for key, value in sample.items()})
samples[0].keys()

In [ ]:
samples[0]['question']

In [ ]:
template = PromptTemplate('perception/design')
# template.ident
print(template.fill(**ctx))

In [ ]:
print(samples[1]['board'])

In [ ]:

tranlation = '''
Lastly, for each sample problem, provide the corresponding JSON formalization based on your schema. Ensure that the JSON representations are valid and adhere to the schema you propose.
'''

rep = '''{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "TicTacToeGameState",
  "description": "Represents the state of a Tic Tac Toe game.",
  "type": "object",
  "properties": {
    "board": {
      "type": "array",
      "description": "The Tic Tac Toe board represented as a 3x3 array. 'X' for Alice, 'O' for the player, and null for empty cells.",
      "items": {
        "type": "array",
        "items": {
          "type": ["string", "null"],
          "enum": ["X", "O", null]
        },
        "minItems": 3,
        "maxItems": 3
      },
      "minItems": 3,
      "maxItems": 3
    },
    "moves": {
      "type": "array",
      "description": "The sequence of moves made in the game, each representing a single placement.",
      "items": {
        "type": "object",
        "properties": {
          "player": {
            "type": "string",
            "enum": ["Alice", "Player"],
            "description": "The player who made the move."
          },
          "row": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "description": "The row index (0-2) where the player made the move."
          },
          "col": {
            "type": "integer",
            "minimum": 0,
            "maximum": 2,
            "description": "The column index (0-2) where the player made the move."
          }
        },
        "required": ["player", "row", "col"]
      }
    },
    "current_player": {
      "type": "string",
      "enum": ["Alice", "Player"],
      "description": "Indicates who's turn it is to play."
    }
  },
  "required": ["board", "moves", "current_player"]
}'''

cand1 = '''{
  "board": [
    [null, null, "X"],
    [null, "O", "X"],
    ["O", null, "O"]
  ],
  "moves": [
    { "player": "Alice", "row": 0, "col": 2 },
    { "player": "Player", "row": 2, "col": 2 },
    { "player": "Alice", "row": 1, "col": 1 },
    { "player": "Player", "row": 1, "col": 0 },
    { "player": "Alice", "row": 0, "col": 0 }
  ],
  "current_player": "Player"
}'''

obs1 = ctx['observation1']


prompt = f'''I want to use this representation to carefully formalize specific problems. The representation uses this schema:
{rep}

For the specific problem:
```
{obs1}
```

The corresponding representation I came up with is:
```json
{cand1}
```
Take a careful look, and if you find any mistakes, please correct them by providing a JSON block with the corrected representation. If you are sure the representation is correct, just say "yes".'''

prompt = f'''I want to use this representation to carefully formalize specific problems. The representation uses this schema:
{rep}

For the specific problem:
```
{obs1}
```

The corresponding representation I came up with is:
```json
{cand1}
```
Take a careful look, and if you find any mistakes, please correct them by providing a JSON block with the corrected representation. If you are sure the representation is correct, just say "yes".'''

prompt = f'''Rephrase the following problem specification in several ways (wrapping each into a markdown code block). Crucially, make sure that all the rephrasings are equivalent, meaning they describe the same problem, but use different wording or structure.

```
{obs1}
```
'''

print(prompt)

In [ ]:
jrep = json.loads(rep)
# jrep

In [ ]:

noisy = ["Alice and I are playing a game on a 3x3 grid. Alice's first move was to the cell in the middle row, right column.  I then played in the bottom row, right column. Alice responded by taking the cell in the bottom row, middle column, and I countered with the middle row, left column. Finally, Alice placed her mark in the top-right corner.", "The game began with Alice playing at (2,3). I then moved to (3,3). Alice followed up by playing at (3,2), and I responded with (2,1). Alice’s last move was to (1,3).", "Player Alice made the first move at the position immediately to the left of the right edge in the center row. Player I responded by claiming the rightmost position in the bottom row. Alice then occupied the center position in the bottom row. I countered by taking the position immediately to the right of the left edge in the middle row. Finally, Alice played in the top-right corner.", "Let's track the game state. Alice's moves were: (2,3), (3,2), (1,3). My moves were: (3,3), (2,1). This represents the sequence of plays so far.", '''The game progressed as follows:
1. Alice: Middle-right
2. I: Bottom-right
3. Alice: Bottom-middle
4. I: Middle-left
5. Alice: Top-right''', "The initial moves in this game are as follows: Alice played to her second choice on the rightmost column, then I played to my second choice on the bottom row, followed by Alice taking the middle of the bottom row, I took the middle-left, and Alice's last play was in the top-right corner." ]

noisy_obs_str = '\n\n'.join([f'```\n{obs}\n```' for i, obs in enumerate(noisy)])

prompt = f'''Using the following schema, carefully formalize the problem described below. The schema is as follows:
{rep}

Here are several (mostly equivalent) ways to describe the problem:

{noisy_obs_str}

Note that although the descriptions vary, they all refer to a description of the same problem, so you should only respond with a single JSON block that captures the essence of the problem as described in the schema. Take care to faithfully represent every aspect of the problem to subsequently solve a reasoning tasks. If minor details conflict between the descriptions, choose the most common interpretation or the one that best fits the schema.
'''
print(prompt)

In [ ]:
prompt = '''Use the following schema to carefully formalize the problem described below. The schema is as follows:
{rep}

For this problem:
```
{nx}
```
'''
# print(prompt)

In [ ]:
nzs = [client.get_response(prompt.format(rep=rep, nx=n), grammar=jrep, temperature=1.)
	   		for n in tqdm(noisy)]
len(nzs)

In [ ]:
boards = []
dats = []
for nz in nzs:
	dat = json.loads(nz)
	dats.append(dat)
	boards.append(dat['board'])
	print(dat)
	print('---'*20)

In [ ]:
boards

In [ ]:
from deepdiff import DeepDiff

In [ ]:
diff = DeepDiff(dats[0], dats[1], ignore_order=False)
print(diff.pretty())

In [ ]:
from jsondiff import diff
json_diff = diff(dats[0], dats[1])

print(json_diff)

In [ ]:
dats[:2]

In [ ]:
obs1 = '''As the adventurer moved along the trail that twisted around the Lonely Mountain, it bent out of sight to the southeast and southwest. The elevation here was higher than to the south, providing a majestic view of the mountain's peak, veiled in a perpetual shroud of clouds. Unfortunately, no visible path from this vantage point led up the mountain.\n\nContinuing the journey, the adventurer arrived at the trail head on the northeast side of the Lonely Mountain. The narrow path snaked through rolling hills. To one side, a sign stood firmly, with a winding path disappearing into the thick undergrowth to the northeast. The trail itself stretched onward to the southeast.\n\nThe trail eventually concluded at Shady Brook, marked by the remnants of a ruined stone foundation-the Old Lingolf House. A small brook meandered lazily by, its gentle flow a contrast to the stillness of the abandoned site. The sun had just climbed above the hills, spilling light over the landscape.\n\nFeeling the weight of the journey, the adventurer filled a jug with water from the brook, quenching both thirst and preparing for the road ahead.\n\nReturning to the trail head, the path was as familiar as it was unwelcoming, flanked by the same rolling hills and the sign that pointed toward the thick undergrowth. The adventurer pressed on, reaching the Eastern Fork where multiple paths diverged around the base of the Lonely Mountain and a smaller path led eastward.\n\nThe road soon led into a small village on the outskirts, with the Lonely Mountain towering ominously in the background. The village path split to the northeast, but no path seemed to climb the mountain directly.\n\nEntering the deserted village revealed echoes of the past. This was once a bustling place where peasants, merchants, and artisans thrived. Now, it was silent except for the smoke rising from a decrepit hovel to the south. The path through the village led from southwest to northeast, and a less-trodden trail headed north up the mountain.\n\nInside the hovel, the air was thick with smoke and the stench of chaos. A pile of rags moved near a bubbling pot, revealing a gnarled hand stirring the contents. The rags belonged to a withered crone who eyed the adventurer with a piercing gaze. \"I should have thought they would send someone more... more...\" She cackled unsettlingly, revealing that everyone had fled from an impending storm in the east. \"Take this and begone!\" With a mystical gesture, she thrust a scroll into the adventurer's hands before pushing them out the door.\n\nBack in the open air, the adventurer unfurled the scroll, which revealed the \"rezrov spell\" capable of opening even locked or enchanted objects. With a sense of urgency, the spell was transcribed into the adventurer's spell book, which glowed with an ethereal light as the words were etched permanently into its pages. The scroll itself vanished in a whisper of magic as the last word was copied.\n\nThe journey resumed, leading back through the village outskirts and on to the Eastern Fork. The paths here were familiar, yet held new promise with the power of the newly acquired spell. The adventurer then ventured onto the Western Road, which wended its way through dark, rolling hills. To the west, a high mountain loomed; to the east, a dark castle with towering spires awaited.'''

In [ ]:
prompt = '''You are a Senior Data Architect specializing in knowledge representation. Your objective is to analyze a sample of unstructured narrative text and design a robust, task-agnostic JSON schema to model its content.

This schema will be the blueprint for converting similar narrative texts into a structured format. The ultimate goal is to support downstream AI tasks such as pathfinding, state tracking, and answering questions about the environment, its characters, and its items.

**Your Task:**

1.  **Analyze the Example Text:** Read the provided text to understand the types of entities and relationships that need to be represented. Pay close attention to:

      * **Locations:** Distinct places the character can be.
      * **Connections:** Paths, roads, or movements between locations, including their direction.
      * **Entities:** Interactive or notable items and characters.
      * **State:** Information that changes, such as the player's inventory, learned knowledge, or environmental conditions.

2.  **Design the Schema:** Based on your analysis, create a generic JSON schema. The schema should:

      * Represent the environment as a navigable graph (i.e., with nodes and edges).
      * Be comprehensive enough to capture all the entity types you identified.
      * Use a system of unique IDs to create stable references between different parts of the data (e.g., connecting a path to its destination location).
      * Be self-documenting. Use the `"description"` field within the schema to explain the purpose of each property.

**Output:**

Your final output must be **only the JSON schema itself**. Do not generate the populated JSON data for the example text.

-----

**Analyze the following example text to inform your schema design:**

```
As the adventurer moved along the trail that twisted around the Lonely Mountain, it bent out of sight to the southeast and southwest. The elevation here was higher than to the south, providing a majestic view of the mountain's peak, veiled in a perpetual shroud of clouds. Unfortunately, no visible path from this vantage point led up the mountain.\n\nContinuing the journey, the adventurer arrived at the trail head on the northeast side of the Lonely Mountain. The narrow path snaked through rolling hills. To one side, a sign stood firmly, with a winding path disappearing into the thick undergrowth to the northeast. The trail itself stretched onward to the southeast.\n\nThe trail eventually concluded at Shady Brook, marked by the remnants of a ruined stone foundation-the Old Lingolf House. A small brook meandered lazily by, its gentle flow a contrast to the stillness of the abandoned site. The sun had just climbed above the hills, spilling light over the landscape.\n\nFeeling the weight of the journey, the adventurer filled a jug with water from the brook, quenching both thirst and preparing for the road ahead.\n\nReturning to the trail head, the path was as familiar as it was unwelcoming, flanked by the same rolling hills and the sign that pointed toward the thick undergrowth. The adventurer pressed on, reaching the Eastern Fork where multiple paths diverged around the base of the Lonely Mountain and a smaller path led eastward.\n\nThe road soon led into a small village on the outskirts, with the Lonely Mountain towering ominously in the background. The village path split to the northeast, but no path seemed to climb the mountain directly.\n\nEntering the deserted village revealed echoes of the past. This was once a bustling place where peasants, merchants, and artisans thrived. Now, it was silent except for the smoke rising from a decrepit hovel to the south. The path through the village led from southwest to northeast, and a less-trodden trail headed north up the mountain.\n\nInside the hovel, the air was thick with smoke and the stench of chaos. A pile of rags moved near a bubbling pot, revealing a gnarled hand stirring the contents. The rags belonged to a withered crone who eyed the adventurer with a piercing gaze. \"I should have thought they would send someone more... more...\" She cackled unsettlingly, revealing that everyone had fled from an impending storm in the east. \"Take this and begone!\" With a mystical gesture, she thrust a scroll into the adventurer's hands before pushing them out the door.\n\nBack in the open air, the adventurer unfurled the scroll, which revealed the \"rezrov spell\" capable of opening even locked or enchanted objects. With a sense of urgency, the spell was transcribed into the adventurer's spell book, which glowed with an ethereal light as the words were etched permanently into its pages. The scroll itself vanished in a whisper of magic as the last word was copied.\n\nThe journey resumed, leading back through the village outskirts and on to the Eastern Fork. The paths here were familiar, yet held new promise with the power of the newly acquired spell. The adventurer then ventured onto the Western Road, which wended its way through dark, rolling hills. To the west, a high mountain loomed; to the east, a dark castle with towering spires awaited.
```

-----'''

print(prompt)

In [ ]:
from ludwig.chess import ChessPuzzle



In [50]:


task = ChessPuzzle(obs_rep='active')
task.prepare()

prob = task.ask_dev(0)
prob.keys()

dict_keys(['puzzle_id', 'fen', 'player', 'legal', 'num_legal_moves', 'question', 'rationale', 'answer', 'system', 'task'])

In [51]:
print(prob['question'])

White just played a2a7. Given the resulting board position:

+---+---+---+---+---+---+---+---+
| R |   |   | K |   |   |   | Q | 1
+---+---+---+---+---+---+---+---+
|   | P | P |   | N | P |   |   | 2
+---+---+---+---+---+---+---+---+
| P |   |   | P |   |   |   |   | 3
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 4
+---+---+---+---+---+---+---+---+
|   |   |   | r | p |   |   |   | 5
+---+---+---+---+---+---+---+---+
|   | p | q |   |   | b |   |   | 6
+---+---+---+---+---+---+---+---+
| p |   | p |   |   |   | p | R | 7
+---+---+---+---+---+---+---+---+
|   | k |   |   |   |   |   | r | 8
+---+---+---+---+---+---+---+---+
  h   g   f   e   d   c   b   a


(where uppercase letters are white pieces and lowercase letters are black pieces)

What is the best move for black? Answer using the UCI or SAN format.


In [54]:

#########

task = TakeTheMiddle(obs_rep='moves')
task.prepare()

prob = task.ask_dev(0)

task = "We're playing tic-tac-toe, and, given the current game state, I would like to know specifically whether or not I should play in the middle next."

obs1 = prob['observation']

##########

# task = ChessPuzzle(obs_rep='active')
# task.prepare()

# prob = task.ask_dev(0)

# task = "We're playing tic-tac-toe, and, given the current game state, I would like to know specifically whether or not I should play in the middle next."

# obs1 = prob['observation']


##########

prompt = f'''We will solve the following task using a structured tree-search framework.

**Task Description:**
{task}

**Example Problem:**
{obs1}

Your first task is to design an optimal state representation for this problem. The representation must be:
1.  **High-Fidelity:** It must capture all details necessary to solve the task unambiguously.
2.  **Efficient:** It should contain minimal redundancy and exclude irrelevant "nuisance" details.
3.  **Conducive to Search:** The structure should make it easy to implement the search functions (`expand`, `extract`, `evaluate`).

Define this representation as a JSON schema, including clear descriptions for each property. Enclose the final output in a ```json ... ``` block.'''

print(prompt)

We will solve the following task using a structured tree-search framework.

**Task Description:**
We're playing tic-tac-toe, and, given the current game state, I would like to know specifically whether or not I should play in the middle next.

**Example Problem:**
Alice started with the top left corner and I played the bottom-middle. She took the middle row left, so I responded with the top middle. Now Alice played at the right middle location.

Your first task is to design an optimal state representation for this problem. The representation must be:
1.  **High-Fidelity:** It must capture all details necessary to solve the task unambiguously.
2.  **Efficient:** It should contain minimal redundancy and exclude irrelevant "nuisance" details.
3.  **Conducive to Search:** The structure should make it easy to implement the search functions (`expand`, `extract`, `evaluate`).

Define this representation as a JSON schema, including clear descriptions for each property. Enclose the final output

In [55]:
# resp = client.get_response(prompt)
chat = client.begin_chat(prompt)

resp = client.step(chat)
rep = json.loads(next(extract_code_blocks(chat[-1]['content'], 'json')))
rep

{'$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'Tic Tac Toe Game State',
 'type': 'object',
 'properties': {'board': {'type': 'array',
   'items': {'type': 'array',
    'items': {'type': 'string', 'enum': ['X', 'O', '']},
    'minItems': 3,
    'maxItems': 3},
   'minItems': 3,
   'maxItems': 3,
   'description': "A 3x3 array representing the game board. 'X' for player X's moves, 'O' for player O's moves, and '' for empty spaces."},
  'turn': {'type': 'string',
   'enum': ['X', 'O'],
   'description': "Whose turn it is. 'X' if it's player X's turn, 'O' if it's player O's turn."}},
 'required': ['board', 'turn'],
 'additionalProperties': False}

In [56]:
prompt = f'''Now your task is to implement the `expand` function in Python. This function takes a single state object (conforming to the schema) and returns a list of all valid next states.

The function should only contain the minimal logic necessary to generate the next states (i.e., the "rules" of the task). Do not include any recursion, search, or heuristic logic within this function.

Provide the implementation as a Python function named `expand` inside a ```python ... ``` block. If you need helper functions, make sure they are defined within the same block so the block can stand alone.'''

chat.append({'role': 'user', 'content': prompt})
resp = client.step(chat)

print(chat[-1]['content'])

pycode = next(extract_code_blocks(chat[-1]['content'], 'python'))

objs = {}
exec(pycode, objs)
expand_fn = objs.get('expand')

objs.keys()

```python
def is_valid_move(board, row, col):
    """Check if a move at position (row, col) is valid."""
    return board[row][col] == ""

def create_next_state(state, row, col):
    """Create a new state by applying a move at position (row, col) to the current state."""
    new_board = [row[:] for row in state["board"]]
    new_board[row][col] = state["turn"]
    new_turn = "O" if state["turn"] == "X" else "X"
    return {"board": new_board, "turn": new_turn}

def expand(state):
    """
    Generate all valid next states from the given state.
    
    Args:
    state (dict): The current state of the game, conforming to the schema.
    
    Returns:
    list: A list of all valid next states.
    """
    next_states = []
    for row in range(3):
        for col in range(3):
            if is_valid_move(state["board"], row, col):
                next_states.append(create_next_state(state, row, col))
    return next_states
```


dict_keys(['__builtins__', 'is_valid_move', 'create_next_state', 'expand'])

In [60]:
prompt = f'''Next, your task is to implement the `extract` function in Python. This function analyzes a search path to find results or decide if the path should be abandoned. Its signature and return values should be:
-   **Input:** Either a single state or a list of states (a trajectory).
-   **Returns `str`:** An extracted result or piece of information. Prune the search here. If the string is not empty, it indicates some useful information has been found to help solve the problem. 
-   **Returns `None`:** No result found yet. The search continues down this path.

First, state whether a **state-based** or **trajectory-based** input is more suitable for this task and briefly explain why.

Then, implement the `extract` function based on your choice. Provide the implementation inside a ```python ... ``` block. If you need helper functions, make sure they are defined within the same block so the block can stand alone.'''

chat.append({'role': 'user', 'content': prompt})
resp = client.step(chat)

pycode = next(extract_code_blocks(chat[-1]['content'], 'python'))
exec(pycode, objs)
extract_fn = objs.get('extract')
# print(chat[-1]['content'])
objs.keys()

dict_keys(['__builtins__', 'is_valid_move', 'create_next_state', 'expand', 'is_winner', 'is_draw', 'extract'])

In [62]:
prompt = f'''To make the search more efficient, we may need a heuristic `evaluate` function.

This function's goal is to guide the search by prioritizing more promising states to solve the problem.

First, state whether a heuristic is necessary for this task.
-   If not, explain why (e.g., the search space is small enough).
-   If it is, explain your chosen approach (e.g., scoring individual states, or sorting a list of candidates) and implement the `evaluate` function.

The two possible approaches are:
1. **Score-based**: The function takes a single state and returns a score indicating its desirability. States with higher scores are considered and expanded first.
2. **Comparison-based**: The function takes a list of states and sorts them in which the most promising states come first. The search will then expand states in this order.

Provide your reasoning and, if applicable, the Python implementation inside a ```python ... ``` block. If you need helper functions, make sure they are defined within the same block so the block can stand alone.'''

chat.append({'role': 'user', 'content': prompt})
resp = client.step(chat)
try:
    pycode = next(extract_code_blocks(chat[-1]['content'], 'python'))
    exec(pycode, objs)
    evaluate_fn = objs.get('evaluate')
except StopIteration:
    evaluate_fn = None
print(chat[-1]['content'])

A heuristic is necessary for this task. Although the search space of a tic-tac-toe game is finite, it is still relatively large, with approximately 255,168 possible unique positions. Without a heuristic, the search could be inefficient, especially if the algorithm has to explore a significant portion of the search space before finding a solution.

I will use a **Score-based** approach. This approach involves assigning a score to each state based on its desirability, with higher scores indicating more promising states. The score can be calculated based on various factors, such as the control of the center, the number of possible wins, and the blocking of the opponent's potential wins.

In the context of the original problem, the goal is to determine whether playing in the middle next is a good move. The heuristic function can be designed to prioritize states where playing in the middle leads to a better position.

```python
def evaluate(state):
    """
    Evaluate the desirability of a

In [42]:
import re

In [ ]:
substrings = ['score-based', 'comparison-based']
pattern = r'\b(?:' + '|'.join(re.escape(sub) for sub in substrings) + r')\b'

text = chat[-1]['content']
matches = re.findall(pattern, text)
matches

[]

In [63]:
prompt = f'''Just to confirm, did you select the **score-based** or **comparison-based** approach for the `evaluate` function? Please respond with only "score-based" or "comparison-based".'''
chat.append({'role': 'user', 'content': prompt})
resp = client.step(chat, grammar=substrings)
print(chat[-1]['content'])

score-based


In [57]:
prompt = f'''Now try to formalize the following example problem using the schema you designed:

** Example Problem: **
```
{obs1}
```'''

chat.append({'role': 'user', 'content': prompt})
resp = client.step(chat, grammar=rep)
z = json.loads(chat[-1]['content'])
z

{'board': [['X', 'X', 'O'], ['O', '', ''], ['', 'X', '']], 'turn': 'X'}

In [30]:
objs['expand'](z)

[{'board': ['X', 'X', None, 'O', 'O', None, None, 'X', None],
  'turn': 'O',
  'is_terminal': False,
  'winner': None},
 {'board': ['X', None, 'X', 'O', 'O', None, None, 'X', None],
  'turn': 'O',
  'is_terminal': False,
  'winner': None},
 {'board': ['X', None, None, 'O', 'O', 'X', None, 'X', None],
  'turn': 'O',
  'is_terminal': False,
  'winner': None},
 {'board': ['X', None, None, 'O', 'O', None, 'X', 'X', None],
  'turn': 'O',
  'is_terminal': False,
  'winner': None},
 {'board': ['X', None, None, 'O', 'O', None, None, 'X', 'X'],
  'turn': 'O',
  'is_terminal': False,
  'winner': None}]

In [58]:
del chat[-2:]

In [59]:
chat

[{'role': 'user',
  'content': 'We will solve the following task using a structured tree-search framework.\n\n**Task Description:**\nWe\'re playing tic-tac-toe, and, given the current game state, I would like to know specifically whether or not I should play in the middle next.\n\n**Example Problem:**\nAlice started with the top left corner and I played the bottom-middle. She took the middle row left, so I responded with the top middle. Now Alice played at the right middle location.\n\nYour first task is to design an optimal state representation for this problem. The representation must be:\n1.  **High-Fidelity:** It must capture all details necessary to solve the task unambiguously.\n2.  **Efficient:** It should contain minimal redundancy and exclude irrelevant "nuisance" details.\n3.  **Conducive to Search:** The structure should make it easy to implement the search functions (`expand`, `extract`, `evaluate`).\n\nDefine this representation as a JSON schema, including clear descriptio